# モデルの実装と学習
この単元では、pytorchを使った**MLPの実装方法**を扱う。  
具体的には、モデルの要素となる**全結合層や活性化関数、損失関数、最適化関数**を実装し、それらを組み合わせた**モデルを定義**する。

これまでで学んだMLPの理論を思い出しつつ、この単元では、MLPモデルの実装方法を学んでいこう！  
この章でモデルの実装方法を学んで、次の章からはモデルを学習していくぞ！

## この単元の目標
- 全結合層と活性化関数ReLUの実装方法を学ぶ
- 損失関数と最適化関数の実装方法を学ぶ
- MLPモデルの実装方法を学ぶ

## 1. 全結合層と活性化関数

In [ ]:
## 本章から必要なモジュールのインポート
import torch
from torch import nn # 全結合層のため
from torch.nn import functional as F # 活性化関数のため

復習になるが、MLPを構成する層は**「全結合層(Fully Connected Layer)」**と呼ばれる。  
pytorchでは、全結合層は`nn.Linear()`クラスでインスタンスとして宣言できる。  
第1引数`in_features`で入力となるノードの数（テンソルのサイズ）、第2引数`out_features`で出力となるノードの数を指定する必要がある。  
また、伝播させるときは`__call__()`メソッドの引数にデータを与えて実行する。

【例題1】 全結合層`fc`を定義して、テンソル`[1,2,3,4]`を伝播させる。



In [ ]:
# 4次元ベクトルを2次元に変換する全結合層をクラスとして定義
fc = nn.Linear(4, 2)
x = torch.Tensor([1,2,3,4])

# fcの__call__()メソッドを呼ぶことでxを伝播できる
x = fc(x)
print(x)

- `tensor([x, y], grad_fn=<AddBackward0>)`と表示できていれば成功だ。  
`［x, y］`には，任意の小数が入る．
- `x, y`は`[1,2,3,4]`を`fc_layer`の初期パラメータで変換した値だ。
- 4次元テンソルが2次元に変換されていることを確認しよう。
- `grad_fn=<AddBackward0>`はパラメータだ。損失を求めるために使う。
- `fc`をインスタンスとして宣言し、`__call__()`で伝播させる流れを理解しよう。

次は、**「活性化関数」**の実装方法を学ぼう。  
pytorchでは，`torch.nn.functional`モジュールに活性化関数が**「関数」**として用意されているので、これを使えばいい。

今回実装するのは、ReLU関数だ。  
これは、入力データに対して「0以下の値を0に変換」する機能を持った活性化関数だ。  

【例題2】 `relu()`関数に、テンソル`[-1.0, -0.5, 0.5, 1.0]`を与えて出力を確認する。

In [ ]:
x = torch.Tensor([-1.0, -0.5, 0.5, 1.0])
x = F.relu(x)
print(x)

- `tensor([0.0000, 0.0000, 0.5000, 1.0000])`と表示されていれば成功だ。
- 0以下の値が「0.0000」となっていることを確認しておこう。


【問題】 テンソル`[-2, -1, 1, 2]`を`nn.Linear()`と`F.relu()`のそれぞれに伝播した結果を表示しよう。  
全結合層の出力ノード数は2とする。

- 全結合層の出力: `tensor([XXXX, XXXX], grad_fn=<AddBackward0>)`
- `relu()`の出力: `tensor([0., 0., 1., 2.])`  
のように表示されていれば正解だ。
- 全結合層の出力となるテンソルの要素は、どんな値でもいい。

## 2. 損失関数と最適化関数

In [ ]:
## 本章から必要なモジュールのインポート
from torch import optim
from torchvision.models import vgg11

損失関数と最適化関数の実装方法について学んでいこう。  
損失関数と最適化関数は、全結合層`fc`のように、  
**「インスタンスを宣言」→「`__call__()`を実行」**  
という流れで使用する。

まずは、**損失関数**だ。  
pytorchでは、第1章でインポートした`torch.nn`モジュールで損失関数が用意されている。  

今回はMSE(平均二乗誤差)を実装する。
MSEは`torch.nn.MSELoss()`というクラスで定義されている。

【例題1】 テンソル`[0,1,2]`とテンソル`[1,-1,0]`のMSEを求める。

In [ ]:
# まず、損失関数をクラスとして用意する
criterion = nn.MSELoss()

x = torch.Tensor([0,1,2])
y = torch.Tensor([1,-1,0])

# 用意したクラスの__call__()を実行
loss = criterion(x, y)

print(loss)

- `tensor(3.)`と表示されていれば成功だ。
- 引数として与えられた2つのテンソルの損失を計算している。
- 要素ごとの差の2乗の平均値となっていることを確認しよう。（下式）
$$ \frac{(0-1)^{2} + (1-(-1))^2 + (2-0)^2}{3} = 3$$

次は、**最適化関数**の実装方法だ。  
pytorchでは`torch.optim`モジュールに最適化関数が用意されている。

最適化関数のインスタンスを宣言するときには、**最適化するモデルのパラメータ**を引数に与える必要があるので、  例として`vgg11`というモデルを用意する。

今回は`Adam`という最適化関数を実装するぞ。  
これは、今でも**頻繁に使われる**実用的なものなので、覚えておこう。  
`Adam`は`torch.optim.Adam()`で定義されている。

【例題2】 `Adam()`のインスタンスを宣言する。

In [ ]:
# モデルを用意する
model = vgg11()

# 用意したモデルのパラメータを与えて、クラスを定義する。
optimizer = optim.Adam(model.parameters())
print(optimizer)

- ```
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
```
と表示されていれば成功だ。
- `lr`は学習率のことで、標準で「0.001」だが、これは引数で変更する事ができる。
- `amsgrad`や`betas`などもハイパーパラメータだ。余裕があれば調べてみよう。

【問題】 損失関数MSEを宣言して、`[1,1,1,1]`と`[0,2,4,6]`の損失を計算、表示してみよう。


- `tensor(9.)`と表示されていれば成功だ。
- 要素ごとの差の2乗の平均値となっていることを確認しよう。（下式）
$$ \frac{(1-0)^{2} + (1-2)^2 + (1-4)^2 + (1-6)^2}{4} = 9$$

## 3. モデルの定義
初期化関数`__init__()`で、モデルに使うレイヤーや関数を準備し、`forward()`関数でモデルを組み立てる。

【例題】 以下の条件のMLPモデルをクラスとして定義する。  
- 「入力層、中間層、出力層」のノードの数が「3、5、2」
- 中間層の出力に活性化関数`relu()`を適用
- 損失関数は`MSELoss()`
- 最適化関数は`Adam()`

In [ ]:
class mlp_net(nn.Module):
    def __init__(self):
        super().__init__()

        # 全結合層を2つ
        self.fc1 = nn.Linear(3, 5)
        self.fc2 = nn.Linear(5, 2)

        # 損失関数と最適化関数
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x):
        x = self.fc1(x)
        print('[fc1を通過]\n', x) # 中間層の出力を表示
        # 活性化関数
        x = F.relu(x)
        print('[relu()を通過]\n', x) # relu()の適用結果を表示
        x = self.fc2(x)
        return x

- モデルクラスを自作するときは`nn.Module`を継承することを忘れないようにしよう。
- 初期化関数`__init__()`について
  - `super().__init__()`により、`nn.Module`の初期化関数を適用する。
  - `forward()`で使用する全結合層と、学習時に使用する損失関数・最適化関数をここで宣言する。
- 順伝播を行う`forward()`について
  - `forward()`は`nn.Module`を継承したことにより`__call__()`メソッドで呼ばれる。
  - ここで、「fc1→relu→fc2」という構造を組み立てている。
  - `x`の値が層を伝播するごとに変化することを認識しよう。


作成したモデルに、テンソル`[0,1,2]`を伝播させてみる。

In [ ]:
# モデルを宣言
model = mlp_net()

x = torch.Tensor([0, 1, 2])

# xを伝播させる
output = model(x)

print('[モデルの出力]\n', output)

- ```
[fc1を通過]
 tensor( {要素が5つのテンソル} , grad_fn=<AddBackward0>)
[relu()を通過]
 tensor( {要素が5つのテンソル} , grad_fn=<ReluBackward0>)
[モデルの出力]
 tensor( {要素が2つのテンソル} , grad_fn=<AddBackward0>)
```  
のように表示されているはずだ。
- fc1, fc2の層ではテンソルの形状が変換され、`relu()`の出力では「0以下の要素が全て0」となっている事を確認しよう。
- モデルの学習を行うときには、この`output`と理想の結果から`self.criterion`と`self.optimizer`を使用して最適化を行う。


【問題】  
例題を参考に、以下の条件のMLPモデルをクラスとして定義してみよう。  
その後、定義したモデルを出力してみよう（例: `class model():`→`model()`で出力）
- 「入力層、中間層、出力層」のノードの数が「4、3、2」
- 中間層の出力に活性化関数`relu()`を適用
- 損失関数は`MSELoss()`
- 最適化関数は`Adam()`

- ```
mlp_net(
  (fc1): Linear(in_features=4, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=2, bias=True)
  (criterion): MSELoss()
)
```
のように表示されていれば成功だ。
- fc1は、全結合層で入力ノードが4つ、出力ノードが3つである。
- fc2は、全結合層で入力ノードが3つ、出力ノードが2つである。
- criterionは、損失関数であり、定義されているのはMSELoss()である。
- 最適化関数は表示されていないが、`{クラス名}.optimizer`で出力できる。  
（例：`mlp_net().optimizer`）


## まとめ
次章のモデルの学習に向けて以下の項目の内容を学んだ。
- 全結合層と活性化関数ReLUの実装方法を学ぶ
- 損失関数と最適化関数の実装方法を学ぶ
- MLPモデルの実装方法を学ぶ

ハイパーパラメータの一つである**モデルの構造**の定義方法について学んだ。

## 次単元の内容
次単元では、個々まで学んだことを全て利用して、モデルの学習・予測を行うぞ！